In [1]:
%pip install plotly
%pip install pandas
import plotly.graph_objects as go
import requests
import json
import datetime
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import time

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pair = "ETHUSDT"
interval = 60

url = f"https://api.kraken.com/0/public/OHLC?pair={pair}&interval={interval}"
response = requests.get(url)
data = json.loads(response.text)

ohlcv_data = data['result'][pair]
times = [datetime.datetime.fromtimestamp(int(entry[0])) for entry in ohlcv_data]
opens = [entry[1] for entry in ohlcv_data]
highs = [entry[2] for entry in ohlcv_data]
lows = [entry[3] for entry in ohlcv_data]
closes = [entry[4] for entry in ohlcv_data]

In [7]:
df = pd.DataFrame(list(zip(times, opens, highs, lows, closes)), columns=['time', 'open', 'high', 'low', 'close'])
df['time'] = pd.to_datetime(df['time'])
df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].apply(pd.to_numeric)

In [8]:
# list of start and end dates
periods = [
    ("2023-04-30 21:00:00", "2023-05-05 14:00:00"),
    ("2023-05-06 04:00:00", "2023-05-12 17:00:00")
]

fig = go.Figure(data=[go.Candlestick(x=df['time'], open=df['open'], high=df['high'], low=df['low'], close=df['close'])])

upper_returns_dict = {}
lower_returns_dict = {}

# loop through each period
for i, (start, end) in enumerate(periods):
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)

    df_filtered = df[(df['time'] >= start_date) & (df['time'] <= end_date)]

    x = np.arange(len(df_filtered))
    y = df_filtered['close'].values
    coeffs = np.polyfit(x, y, 1)

    middle_line = coeffs[0] * x + coeffs[1]
    std = np.std(df_filtered['close'].values)
    upper_line = middle_line + std
    lower_line = middle_line - std

    # Iterate through the filtered dataframe
    for j in range(len(df_filtered)):
        # Calculate the distance of the candle from the trendline
        distance_upper = df_filtered.iloc[j]['high'] - upper_line[j]
        distance_lower = df_filtered.iloc[j]['low'] - lower_line[j]

        # If the distance is within 0.005 (0.5%) of the trendline
        if abs(distance_upper) <= upper_line[j]*0.005:
            # Calculate returns for all subsequent candles
            returns = (df_filtered.iloc[j+1:]['close'] - df_filtered.iloc[j]['close']) / df_filtered.iloc[j]['close']
            # Store returns in the dictionary with the index of the touch event as the key
            upper_returns_dict[(i, j)] = returns

        elif abs(distance_lower) <= lower_line[j]*0.005:
            # Calculate returns for all subsequent candles
            returns = (df_filtered.iloc[j+1:]['close'] - df_filtered.iloc[j]['close']) / df_filtered.iloc[j]['close']
            # Store returns in the dictionary with the index of the touch event as the key
            lower_returns_dict[(i, j)] = returns

    slope = coeffs[0]
    y_int = coeffs[1]

    print(f"\nTrend Line {i+1}:")
    print(f"Slope of Middle Line: {slope:.4f}$/Candle")
    print(f"y intercept: {y_int:.2f}$")
    print(f"standard deviation: {std:.2f}$")
    print("First 5 points of upper line:", upper_line[:5])
    print("First 5 points of middle line:", middle_line[:5])
    print("First 5 points of lower line:", lower_line[:5])

    fig.add_trace(go.Scatter(x=df_filtered['time'], y=middle_line, mode='lines', name=f'Middle Line {i+1}', line=dict(dash= 'dot', color='blue')))
    fig.add_trace(go.Scatter(x=df_filtered['time'], y=upper_line, mode='lines', name=f'Upper Line {i+1}', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df_filtered['time'], y=lower_line, mode='lines', name=f'Lower Line {i+1}', line=dict(color='green')))

    fig.update_layout(title='ETH/USD OHLC Data', yaxis_title='ETH/USD')
    
fig.show()


Trend Line 1:
Slope of Middle Line: 0.8802$/Candle
y intercept: 1821.01$
standard deviation: 34.73$
First 5 points of upper line: [1855.7337711  1856.61392651 1857.49408191 1858.37423732 1859.25439272]
First 5 points of middle line: [1821.00850038 1821.88865579 1822.76881119 1823.64896659 1824.529122  ]
First 5 points of lower line: [1786.28322966 1787.16338506 1788.04354047 1788.92369587 1789.80385127]

Trend Line 2:
Slope of Middle Line: -0.8859$/Candle
y intercept: 1916.51$
standard deviation: 44.69$
First 5 points of upper line: [1961.19368555 1960.30780214 1959.42191873 1958.53603533 1957.65015192]
First 5 points of middle line: [1916.50634106 1915.62045765 1914.73457424 1913.84869084 1912.96280743]
First 5 points of lower line: [1871.81899656 1870.93311316 1870.04722975 1869.16134635 1868.27546294]


In [9]:
fig = go.Figure()

# Iterate through each touch event in the dictionaries and add a line for the values over time
for key, values in lower_returns_dict.items():
    period_idx, touch_idx = key
    fig.add_trace(go.Scatter(y=values.values, mode='lines', 
                             name=f'Lower Touch Event at {touch_idx} (Period {period_idx+1})', line=dict(color='green')))

for key, values in upper_returns_dict.items():
    period_idx, touch_idx = key
    fig.add_trace(go.Scatter(y=values.values, mode='lines', 
                             name=f'Upper Touch Event at {touch_idx} (Period {period_idx+1})', line=dict(color='red')))

fig.update_layout(title='Returns Over Time After Touch Event', xaxis_title='Candles after touch event', yaxis_title='Returns')
fig.show()